In [2]:
import fitz
import re
import string
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix,classification_report,roc_curve
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [3]:
# PDF faylını oxumaq
cv_files = ['alfred_pennyworth_pm.pdf', 'barry_allen_fe.pdf','bruce_wayne_fullstack.pdf','john_doe.pdf','harvey_dent_mle.pdf']
df_cv = pd.DataFrame(columns=['cv'])

for cv in cv_files:
    file_path = f'C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\Resumes\\{cv}'

    document = fitz.open(file_path)
    text = ""
    # Hər səhifənin mətnini oxumaq
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
        
    df_cv = df_cv.append({'cv': text}, ignore_index=True)
  
df_cv

C:\Users\Edish\AppData\Local\Temp\ipykernel_984\819538062.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cv = df_cv.append({'cv': text}, ignore_index=True)


FileNotFoundError: no such file: 'C:\Users\Edish\Documents\Python Taskilled\Project Mentorness\Resumes\barry_allen_fe.pdf'

In [ ]:
# PDF faylını oxumaq
job_files = ['job_desc_front_end_engineer.pdf', 'job_desc_full_stack_engineer.pdf','job_desc_java_developer.pdf','job_desc_product_manager.pdf','ml vacancies.pdf']
df_job = pd.DataFrame(columns=['job'])

for job in job_files:
    file_path = f'C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\JobDescription\\{job}'


    document = fitz.open(file_path)
    text = ""
    # Hər səhifənin mətnini oxumaq
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
        
    df_job = df_job.append({'job': text}, ignore_index=True)
  
df_job

In [ ]:
# Stopwords yüklə

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:

# Mətn təmizləmə funksiyası (stop words çıxarılması daxil)
def clean_text_cv(text):
    text = text.lower()
    text = re.sub(r'\b\d{3,}\b', '', text)
    text = re.sub(r'[^\w\s.]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    match = re.search(r'(objective|professional summary)', text, re.IGNORECASE)
    if text.strip()[-1].isdigit():
        text = re.sub(r'\d+\s*$', '', text).strip()
    if match:
        return text[match.end():]  # Başlıqdan sonrakı mətn hissəsini qaytar
    return text  

    # Stop words sil
    text = ' '.join(word for word in text.split() if word not in stop_words)

    # Lemmatization
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())    

def clean_text_job(text):
    text = text.lower()
    text = re.sub(r'\b\d{3,}\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    match = re.search(r'(how to apply)', text, re.IGNORECASE)
    if match:
        return text[:match.start()]  # Başlıqdan əvvəlki mətn hissəsini qaytar
    return text  # Əgər başlıq tapılmazsa, bütün mətni qaytar

    # Stop words sil
    text = ' '.join(word for word in text.split() if word not in stop_words)

    # Lemmatization
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())


df_cv['clean_cv'] = df_cv['cv'].apply(clean_text_cv)
df_job['clean_job'] = df_job['job'].apply(clean_text_job)


In [ ]:
df_job['clean_job'][4]

In [ ]:
df_cv['clean_cv'][0]

In [ ]:
cv_texts = df_cv['clean_cv'].tolist()
job_texts = df_job['clean_job'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Modeli yükləyin
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


# CV və iş təsvirlərini vektorlaşdirin
cv_embeddings = model.encode(cv_texts, convert_to_tensor=True)
job_embeddings = model.encode(job_texts, convert_to_tensor=True)

# Cosine Similarity hesablayın
similarity_scores = util.pytorch_cos_sim(cv_embeddings, job_embeddings)
print("Cosine Similarity Scores with BERT:", similarity_scores)

In [1]:
similarity_scores = similarity_scores.reshape(-1,1)
similarity_scores[0]

NameError: name 'similarity_scores' is not defined

In [133]:
threshold = 0.5  # Uyğunluq skoru üçün threshold
y = []

# Cosine similarity matrisindən etiketi təyin etmək
for row in similarity_scores:
    row_labels = [1 if score >= threshold else 0 for score in row]
    y.extend(row_labels)

y = np.array(y)
print(y)


[1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1]


In [134]:
import numpy as np
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(similarity_scores, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)


torch.Size([20, 1]) torch.Size([5, 1])


In [135]:
def evaluate_model_performance(model = None,train_features = None,train_labels = None,test_features = None,test_labels =None,algorithm_name = None):
    
    accuracy = accuracy_score(y_true = test_labels, y_pred = model.predict(X = test_features))
    precision = precision_score(y_true = test_labels, y_pred = model.predict(X = test_features))
    recall = recall_score(y_true = test_labels, y_pred = model.predict(X = test_features))
    f1 = f1_score(y_true= test_labels, y_pred= model.predict(X = test_features))
    auc = roc_auc_score(y_true= test_labels, y_score = model.predict(X = test_features))
    
    print(f"Model Performance for:{algorithm_name}\n")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("AUC:", auc)

In [136]:
lg_classifier = LogisticRegression()
lg_classifier = lg_classifier.fit(X_train, y_train)

model_summary = evaluate_model_performance(model = lg_classifier, 
                                              train_features = X_train, 
                                              train_labels = y_train, 
                                              test_features = X_test, 
                                              test_labels = y_test,
                                              algorithm_name = 'Logistic Regression')
model_summary

Model Performance for:Logistic Regression

Accuracy: 0.8
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5


C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [137]:
# SVM modelinin qurulması
svm_model = SVC(kernel='linear')

# Modelin təlimi
svm_model = svm_model.fit(X_train, y_train)

model_summary = evaluate_model_performance(model = svm_model, 
                                              train_features = X_train, 
                                              train_labels = y_train, 
                                              test_features = X_test, 
                                              test_labels = y_test,
                                              algorithm_name = 'SVM model')
model_summary

Model Performance for:SVM model

Accuracy: 0.8
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5


C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [138]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier = xgb_classifier.fit(X_train, y_train)

model_summary = evaluate_model_performance(model = xgb_classifier, 
                                              train_features = X_train, 
                                              train_labels = y_train, 
                                              test_features = X_test, 
                                              test_labels = y_test,
                                              algorithm_name = 'XGB Model')
model_summary

Model Performance for:XGB Model

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
AUC: 1.0


In [140]:
with open(file = 'best_model.pickle',mode = 'wb') as pickled_file:
    pickle.dump(obj = svm_model,file = pickled_file)

In [54]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import normalize

# # TF-IDF dəyərlərini əldə edin
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(cv_texts + job_texts)
# words = vectorizer.get_feature_names_out()

# # Sözlərə uyğun vektorları alın
# def get_tfidf_weighted_vector(text, model, tfidf, words):
#     words = text.split()
#     vectors = [model.wv[word] * tfidf[words.index(word)] for word in words if word in model.wv and word in words]
#     return np.mean(vectors, axis=0)

# # Vektorları əldə edin
# tfidf_weights = tfidf_matrix.toarray()
# weighted_vectors = [get_tfidf_weighted_vector(text, model, tfidf_weights[i], words) for i, text in enumerate(cv_texts + job_texts)]

# # Cosine Similarity hesablayın
# similarity_scores = cosine_similarity(weighted_vectors[:len(cv_texts)], weighted_vectors[len(cv_texts):])
# print("TF-IDF Weighted Cosine Similarity Scores:", similarity_scores)


TF-IDF Weighted Cosine Similarity Scores: [[0.5499819  0.52754265 0.67972195 0.4689391  0.5760403 ]
 [0.5874162  0.4211997  0.4903461  0.28018028 0.5129319 ]
 [0.5718806  0.5870232  0.5381878  0.41839695 0.6218357 ]
 [0.4910674  0.42190823 0.5645016  0.43982476 0.5834123 ]
 [0.4977513  0.5088762  0.5290378  0.34404066 0.60508555]]


In [52]:
# # CV və iş təsvirlərini list şəklinə çevirin
# cv_texts = df_cv['clean_cv'].tolist()
# job_texts = df_job['clean_job'].tolist()

# # TfidfVectorizer obyektini yaradın
# vectorizer = TfidfVectorizer()

# # CV və iş təsvirlərini birləşdirin və uyğunlaşdırın
# combined_texts = cv_texts + job_texts
# combined_tfidf = vectorizer.fit_transform(combined_texts)

# # CV və iş təsvirləri üçün vektorları ayırın
# cv_tfidf = combined_tfidf[:len(cv_texts)]
# job_tfidf = combined_tfidf[len(cv_texts):]

# #cv_tfidf = cv_tfidf.toarray()
# #job_tfidf = job_tfidf.toarray()
# # CV və iş təsvirləri arasında oxşarlığı hesablayırıq
# similarity_scores = cosine_similarity(cv_tfidf, job_tfidf)

# # Nəticələri göstərmək
# print(similarity_scores)

[[0.2713236  0.2952416  0.34659774 0.38327186 0.29512025]
 [0.25405963 0.19422723 0.21242943 0.15701689 0.21376736]
 [0.30882144 0.33344599 0.34866195 0.32795125 0.34520656]
 [0.29494089 0.32418303 0.44685011 0.27817898 0.30080672]
 [0.20852865 0.21122893 0.22946584 0.22655039 0.49530609]]


### Bert Model

In [56]:
# from transformers import BertTokenizer, BertModel

# # ALBERT tokenləşdirmə metodunu yükləyin
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# cv_texts = df_cv['clean_cv'].tolist()
# job_texts = df_job['clean_job'].tolist()

# # CV və vakansiya mətnlərini tokenləşdirin
# cv_tokens = tokenizer(cv_texts, return_tensors='pt', padding=True, truncation=True)
# job_tokens = tokenizer(job_texts, return_tensors='pt', padding=True, truncation=True)

In [58]:
# from transformers import BertModel
# import torch

# # BERT modelini yükləyin
# bert_model = BertModel.from_pretrained('bert-base-uncased')

# # CV və vakansiya tokenlərini BERT modelinə verərək vektorlara çevirin
# with torch.no_grad():
#     cv_outputs = bert_model(**cv_tokens)
#     job_outputs = bert_model(**job_tokens)

# # CV və vakansiya vektorlarını əldə edin
# cv_embedding = cv_outputs.last_hidden_state.mean(dim=1)
# job_embedding = job_outputs.last_hidden_state.mean(dim=1)

In [59]:
# from sklearn.metrics.pairwise import cosine_similarity

# # CV və vakansiya vektorları arasında kosinus bənzərliyini hesablamaq
# similarity = cosine_similarity(cv_embedding, job_embedding)

# print("Uyğunluq dərəcəsi:", similarity)


Uyğunluq dərəcəsi: [[0.94543344 0.9649761  0.9687836  0.967291   0.9602374 ]
 [0.9631349  0.9719155  0.9696667  0.9667721  0.9264314 ]
 [0.93799853 0.953841   0.9735334  0.9519393  0.9451593 ]
 [0.958897   0.9707974  0.9795161  0.9594753  0.9303658 ]
 [0.9550263  0.97017306 0.97079575 0.9645685  0.9495796 ]]


### Albert Model

In [23]:
# from transformers import AlbertTokenizer, AlbertModel

# # ALBERT tokenləşdirmə metodunu yükləyin
# tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# cv_texts = df_cv['clean_cv'].tolist()
# job_texts = df_job['clean_job'].tolist()

# # CV və vakansiya mətnlərini tokenləşdirin
# cv_tokens = tokenizer(cv_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
# job_tokens = tokenizer(job_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)


In [24]:
# from transformers import AlbertModel
# import torch

# # BERT modelini yükləyin
# model = AlbertModel.from_pretrained('albert-base-v2')

# # CV və vakansiya tokenlərini BERT modelinə verərək vektorlara çevirin
# with torch.no_grad():
#     cv_outputs = model(**cv_tokens)
#     job_outputs = model(**job_tokens)

# # CV və vakansiya vektorlarını əldə edin
# cv_embedding = cv_outputs.last_hidden_state.mean(dim=1)
# job_embedding = job_outputs.last_hidden_state.mean(dim=1)


In [25]:
# from sklearn.metrics.pairwise import cosine_similarity

# # CV və vakansiya vektorları arasında kosinus bənzərliyini hesablamaq
# similarity = cosine_similarity(cv_embedding, job_embedding)

# print("Uyğunluq dərəcəsi:", similarity)


Uyğunluq dərəcəsi: [[0.9538876  0.97145283 0.9490682  0.9583086 ]
 [0.9093657  0.9410251  0.87654054 0.91929376]
 [0.95489985 0.92184323 0.95910424 0.89633787]
 [0.94784176 0.9595026  0.9431582  0.9293703 ]
 [0.9348298  0.9496388  0.92542815 0.9364503 ]]


### Word2Vec model

In [53]:
# from gensim.models import Word2Vec

# # Mətni cümlələrə böl
# sentences = [text.split() for text in cv_texts + job_texts]

# # Word2Vec modelini öyrət
# model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Vektorizasiya funksiyası
# def get_vector(text):
#     words = text.split()
#     vector = [model.wv[word] for word in words if word in model.wv]
#     return np.mean(vector, axis=0)

# # Vektorları əldə et
# cv_vectors = np.array([get_vector(text) for text in cv_texts])
# job_vectors = np.array([get_vector(text) for text in job_texts])

# # Cosine Similarity hesablayın
# similarity_scores = cosine_similarity(cv_vectors, job_vectors)
# print("Cosine Similarity Scores with Word2Vec:", similarity_scores)


Cosine Similarity Scores with Word2Vec: [[0.9556517  0.94924647 0.96595997 0.94669056 0.9490228 ]
 [0.9394299  0.93133664 0.9341774  0.91734827 0.9365722 ]
 [0.9540197  0.954368   0.9597762  0.9509833  0.9524786 ]
 [0.9502     0.9455668  0.9580271  0.9345489  0.9474186 ]
 [0.9442921  0.9484159  0.9499204  0.94843334 0.95516396]]
